# Логистическая регрессия. Практика

В этом задании вам предлагается спрогнозировать, купит клиент велосипед или нет, обучив логистическую регрессию.

In [1]:
# подключим библиотеки
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import warnings
warnings.filterwarnings("ignore")

In [2]:
# считаем данные
data = pd.read_csv('https://raw.githubusercontent.com/evgpat/edu_stepik_practical_ml/main/datasets/bike_buyers_clean.csv')

In [3]:
# выводим первые 5 строк датафрейма 
data.head(5)

,ID,Marital Status,Gender,Income,Children,Education,Occupation,Home Owner,Cars,Commute Distance,Region,Age,Purchased Bike
0,12496,Married,Female,40000,1,Bachelors,Skilled Manual,Yes,0,0-1 Miles,Europe,42,No
1,24107,Married,Male,30000,3,Partial College,Clerical,Yes,1,0-1 Miles,Europe,43,No
2,14177,Married,Male,80000,5,Partial College,Professional,No,2,2-5 Miles,Europe,60,No
3,24381,Single,Male,70000,0,Bachelors,Professional,Yes,1,5-10 Miles,Pacific,41,Yes
4,25597,Single,Male,30000,0,Bachelors,Clerical,No,0,0-1 Miles,Europe,36,Yes


In [ ]:
# смотрим размер датафрейма
data.shape

(1000, 13)

Выведите статистики по категориальным признакам, чтобы посмотреть, сколько категорий в каждом категориальном (нечисловом) признаке.

Для этого можно воспользоваться методом `describe` из библиотеки pandas со значением параметра `include = 'object'`.

**Вопрос:** в каком категориальном признаке встречаются три различных значения?

In [ ]:
data.describe(include='object')

,Marital Status,Gender,Education,Occupation,Home Owner,Commute Distance,Region,Purchased Bike
count,1000,1000,1000,1000,1000,1000,1000,1000
unique,2,2,5,5,2,5,3,2
top,Married,Male,Bachelors,Professional,Yes,0-1 Miles,North America,No
freq,539,509,306,276,685,366,508,519


Закодируйте все категориальные столбцы с двумя категориями следующим образом:  
самая часто встречающаяся категория превращается в 1, другая в 0.

In [4]:
data['Marital Status'] = data['Marital Status'].map({'Married': 1, 'Single': 0})

In [5]:
data['Gender'] = data['Gender'].map({'Male': 1, 'Female': 0})

In [6]:
data['Home Owner'] = data['Home Owner'].map({'Yes': 1, 'No': 0})

In [7]:
data['Purchased Bike'] = data['Purchased Bike'].map({'No': 1, 'Yes': 0})

Удалите остальные категориальные столбцы.

**Вопрос:** сколько категориальных столбцов вы удалили?

In [8]:
data.drop(['Education', 'Occupation', 'Commute Distance', 'Region'], axis=1, inplace=True)

In [ ]:
data

,ID,Marital Status,Gender,Income,Children,Home Owner,Cars,Age,Purchased Bike
0,12496,1,0,40000,1,1,0,42,1
1,24107,1,1,30000,3,1,1,43,1
2,14177,1,1,80000,5,0,2,60,1
3,24381,0,1,70000,0,1,1,41,0
4,25597,0,1,30000,0,0,0,36,0
...,...,...,...,...,...,...,...,...,...
995,23731,1,1,60000,2,1,2,54,0
996,28672,0,1,70000,4,1,0,35,0
997,11809,1,1,60000,2,1,0,38,0
998,19664,0,1,100000,3,0,3,38,1


Удалите столбец `ID`, так как он по сути нечисловой.

In [9]:
data.drop('ID', axis=1, inplace=True)

Сформируйте матрицу объект-признак `X` и вектор `y` с целевой переменной.  
Целевая переменная - это последний столбец, `Purchased Bike`.

In [10]:
X = data.iloc[:,:7]

In [11]:
y = data.iloc[:,-1]

In [ ]:
X.shape, y.shape

((1000, 7), (1000,))

Разбейте данные на тренировочную и тестовую часть (`Xtrain`, `Xtest`, `ytrain`, `ytest`), в тест отправьте 30% данных.  
Зафиксируйте `random_state = 42`.

In [27]:
from sklearn.model_selection import train_test_split

Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, train_size=0.7, random_state=42)

**Вопрос:** сколько объектов в матрице `Xtrain`?

In [ ]:
Xtrain.shape

(700, 7)

Почти всё готово для обучения модели!

Осталось отмасштабировать матрицу `X`, так как линейные модели чувствительны к масштабу данных.

*  Обучите на тренировочной матрице (`Xtrain`) `MinMaxScaler` из библиотеки `sklearn.preprocessing`
*  Примените масштабирование и к `Xtrain`, и к `Xtest`
*  Переведите полученные после масштабирования `np.array` обратно в pandas `dataframe`.

Полученные масштабированные матрицы назовите, как и раньше, `Xtrain` и `Xtest`.

In [28]:
from sklearn.preprocessing import MinMaxScaler

Scaler = MinMaxScaler().fit(Xtrain)

In [ ]:
Xtrain.columns

Index(['Marital Status', 'Gender', 'Income', 'Children', 'Home Owner', 'Cars',
       'Age'],
      dtype='object')

In [29]:
Xtrain = pd.DataFrame(Scaler.transform(Xtrain), columns=Xtrain.columns)

In [30]:
Xtest = pd.DataFrame(Scaler.transform(Xtest), columns=Xtest.columns)

In [31]:
Xtest

,Marital Status,Gender,Income,Children,Home Owner,Cars,Age
0,0.0,1.0,0.1875,0.8,1.0,0.50,0.578125
1,1.0,1.0,0.3750,0.4,0.0,0.50,0.375000
2,1.0,1.0,0.1875,0.8,0.0,0.00,0.078125
3,1.0,0.0,0.3125,0.2,1.0,0.00,0.171875
4,1.0,1.0,0.3750,1.0,1.0,0.50,0.281250
...,...,...,...,...,...,...,...
295,1.0,0.0,0.4375,1.0,1.0,0.75,0.234375
296,1.0,0.0,0.3125,0.2,1.0,0.25,0.312500
297,1.0,1.0,0.6250,0.8,1.0,0.75,0.359375
298,1.0,1.0,0.1875,0.2,1.0,0.00,0.281250


In [32]:
Xtrain

,Marital Status,Gender,Income,Children,Home Owner,Cars,Age
0,1.0,1.0,0.2500,0.2,1.0,0.00,0.140625
1,0.0,1.0,0.5000,0.0,0.0,0.75,0.140625
2,0.0,1.0,0.1875,0.0,0.0,0.00,0.171875
3,0.0,1.0,0.6250,0.0,0.0,0.75,0.109375
4,1.0,1.0,0.5000,1.0,1.0,0.75,0.250000
...,...,...,...,...,...,...,...
695,1.0,1.0,0.3750,0.4,1.0,0.50,0.421875
696,0.0,0.0,0.0000,0.4,1.0,0.00,0.406250
697,0.0,1.0,0.1250,0.0,1.0,0.25,0.109375
698,0.0,0.0,0.0000,0.4,0.0,0.25,0.671875


Теперь обучите логистическую регрессию на тренировочных данных

In [33]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression().fit(Xtrain, ytrain)

Сделайте предсказания на тренировочных и на тестовых данных.

In [34]:
prediction_train = model.predict(Xtrain)
prediction_test = model.predict(Xtest)

In [35]:
prediction_test

array([1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0,
       1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1,
       1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0,
       1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1,
       0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1,
       0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0,
       0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1,
       1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0,
       1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1,
       1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0,
       0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0,
       1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1])

Оцените значение accuracy на трейне и на тесте.

In [36]:
from sklearn.metrics import accuracy_score

model_score_train = accuracy_score(ytrain, prediction_train)
model_score_test = accuracy_score(ytest, prediction_test)

In [37]:
model_score_train, model_score_test

(0.6342857142857142, 0.58)

Качество модели получилось невысоким, зато модель не переобучена.

Попробуем добавить новых признаков в модель, используя [PolynomialFeatures](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.PolynomialFeatures.html).

Создайте полиномиальные признаки degree = 2.

Как обычно:
*  `fit` делайте на тренировочных данных
*  `transform` и на тренировочных, и на тестовых данных. Затем верните результат к формату pandas `dataframe`.

Полученные матрицы назовите, как и раньше, `Xtrain` и `Xtest`.

In [38]:
from sklearn.preprocessing import PolynomialFeatures

Poly = PolynomialFeatures().fit(Xtrain)

Xtrain = pd.DataFrame(Poly.transform(Xtrain))
Xtest = pd.DataFrame(Poly.transform(Xtest))

In [39]:
Xtrain

,0,1,2,3,4,5,6,7,8,9,...,26,27,28,29,30,31,32,33,34,35
0,1.0,1.0,1.0,0.2500,0.2,1.0,0.00,0.140625,1.0,1.0,...,0.04,0.2,0.00,0.028125,1.0,0.00,0.140625,0.0000,0.000000,0.019775
1,1.0,0.0,1.0,0.5000,0.0,0.0,0.75,0.140625,0.0,0.0,...,0.00,0.0,0.00,0.000000,0.0,0.00,0.000000,0.5625,0.105469,0.019775
2,1.0,0.0,1.0,0.1875,0.0,0.0,0.00,0.171875,0.0,0.0,...,0.00,0.0,0.00,0.000000,0.0,0.00,0.000000,0.0000,0.000000,0.029541
3,1.0,0.0,1.0,0.6250,0.0,0.0,0.75,0.109375,0.0,0.0,...,0.00,0.0,0.00,0.000000,0.0,0.00,0.000000,0.5625,0.082031,0.011963
4,1.0,1.0,1.0,0.5000,1.0,1.0,0.75,0.250000,1.0,1.0,...,1.00,1.0,0.75,0.250000,1.0,0.75,0.250000,0.5625,0.187500,0.062500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
695,1.0,1.0,1.0,0.3750,0.4,1.0,0.50,0.421875,1.0,1.0,...,0.16,0.4,0.20,0.168750,1.0,0.50,0.421875,0.2500,0.210938,0.177979
696,1.0,0.0,0.0,0.0000,0.4,1.0,0.00,0.406250,0.0,0.0,...,0.16,0.4,0.00,0.162500,1.0,0.00,0.406250,0.0000,0.000000,0.165039
697,1.0,0.0,1.0,0.1250,0.0,1.0,0.25,0.109375,0.0,0.0,...,0.00,0.0,0.00,0.000000,1.0,0.25,0.109375,0.0625,0.027344,0.011963
698,1.0,0.0,0.0,0.0000,0.4,0.0,0.25,0.671875,0.0,0.0,...,0.16,0.0,0.10,0.268750,0.0,0.00,0.000000,0.0625,0.167969,0.451416


In [40]:
Xtest

,0,1,2,3,4,5,6,7,8,9,...,26,27,28,29,30,31,32,33,34,35
0,1.0,0.0,1.0,0.1875,0.8,1.0,0.50,0.578125,0.0,0.0,...,0.64,0.8,0.40,0.462500,1.0,0.50,0.578125,0.2500,0.289062,0.334229
1,1.0,1.0,1.0,0.3750,0.4,0.0,0.50,0.375000,1.0,1.0,...,0.16,0.0,0.20,0.150000,0.0,0.00,0.000000,0.2500,0.187500,0.140625
2,1.0,1.0,1.0,0.1875,0.8,0.0,0.00,0.078125,1.0,1.0,...,0.64,0.0,0.00,0.062500,0.0,0.00,0.000000,0.0000,0.000000,0.006104
3,1.0,1.0,0.0,0.3125,0.2,1.0,0.00,0.171875,1.0,0.0,...,0.04,0.2,0.00,0.034375,1.0,0.00,0.171875,0.0000,0.000000,0.029541
4,1.0,1.0,1.0,0.3750,1.0,1.0,0.50,0.281250,1.0,1.0,...,1.00,1.0,0.50,0.281250,1.0,0.50,0.281250,0.2500,0.140625,0.079102
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,1.0,1.0,0.0,0.4375,1.0,1.0,0.75,0.234375,1.0,0.0,...,1.00,1.0,0.75,0.234375,1.0,0.75,0.234375,0.5625,0.175781,0.054932
296,1.0,1.0,0.0,0.3125,0.2,1.0,0.25,0.312500,1.0,0.0,...,0.04,0.2,0.05,0.062500,1.0,0.25,0.312500,0.0625,0.078125,0.097656
297,1.0,1.0,1.0,0.6250,0.8,1.0,0.75,0.359375,1.0,1.0,...,0.64,0.8,0.60,0.287500,1.0,0.75,0.359375,0.5625,0.269531,0.129150
298,1.0,1.0,1.0,0.1875,0.2,1.0,0.00,0.281250,1.0,1.0,...,0.04,0.2,0.00,0.056250,1.0,0.00,0.281250,0.0000,0.000000,0.079102


**Вопрос:** на сколько признаков стало больше при добавлении полиномиальных признаков второй степени?

In [ ]:
36-7

29

Заново обучите логистическую регрессию, уже на расширенной матрице признаков, и сделайте предсказания на трейне и тесте, а затем оцените качество (*accuracy*).

**Вопрос:** на сколько повысилось качество модели на тестовых данных?  
Ответ округлите до сотых.

In [41]:
model = LogisticRegression().fit(Xtrain, ytrain)
prediction_train = model.predict(Xtrain)
prediction_test = model.predict(Xtest)

In [42]:
model_score_train = accuracy_score(ytrain, prediction_train)
model_score_test = accuracy_score(ytest, prediction_test)

In [43]:
model_score_train, model_score_test

(0.6871428571428572, 0.6233333333333333)

In [44]:
round((model_score_test - 0.58), 2)

0.04

Появились новые требования от заказчика!

Заказчик просит, чтобы:
*  доля найденных моделью потенциальных покупателей была максимальной 
*  accuracy при этом была не ниже, чем 0.6 (отклонения accuracy на тестовых данных на $\pm 0.05$ допустимы).

Сначала посмотрите, какие значения recall и accuracy имеют предсказния модели на тесте с классами, предсказанными по умолчанию (методом `predict`).

In [45]:
from sklearn.metrics import recall_score, precision_score

In [46]:
recall = recall_score(ytrain, prediction_train)
precision = precision_score(ytrain, prediction_train)

In [47]:
recall, precision

(0.738544474393531, 0.6919191919191919)

Подберём порог для перевода вероятностей в классы, чтобы оптимизировать требуемые метрики!

Разобъем тренировочные данные на трейн и валидацию, чтобы по валидационной части подбирать порог.

In [48]:
XtrainS, Xval, ytrainS, yval = train_test_split(Xtrain, ytrain, test_size=0.3, random_state=42)

XtrainS.shape, Xval.shape

((490, 36), (210, 36))

* Обучите модель на тренировочных данных.
* Предскажите вероятности положительного класса на валидационных данных

В цикле для каждого значения порога:
*  переведите вероятности в классы
*  вычислите полноту (на валидационных данных)

Выведите на экран:

1) значение порога, дающее максимальный *recall*, при условии, что accuracy $\geq$ 0.6.

2) значение *recall* при этом пороге

3) значение *accuracy* для этого порога


Ищите порог на отрезке от 0 до 1 с шагом 0.01.

In [49]:
model_final = LogisticRegression().fit(XtrainS, ytrainS)

In [52]:
pred_probability = model_final.predict_proba(Xval)[:, 1]

In [53]:
from sklearn.metrics import recall_score

RecMax = -1
BestThr = -1
BestAcc = -1

for thr in np.arange(0, 1, 0.01):
    pred_class = pred_probability >= thr
    new_recall = recall_score(yval, pred_class)
    new_accuracy = accuracy_score(yval, pred_class)
    if new_accuracy >= 0.6 and new_recall > RecMax:
      RecMax = new_recall
      BestThr = thr
      BestAcc = new_accuracy
print (BestThr, RecMax, BestAcc)

0.42 0.8518518518518519 0.6142857142857143


In [65]:
# for thr in np.arange(0, 1, 0.01):
#     pred_class = pred_probability >= thr
#     new_recall = recall_score(yval, pred_class)
#     new_accuracy = accuracy_score(yval, pred_class)
#     print(new_recall, new_accuracy)

Теперь заново обучите модель на исходных тренировочных данных (`Xtrain`, `ytrain`), предскажите вероятности на тесте и переведите их в классы по найденному порогу.

In [55]:
final = LogisticRegression().fit(Xtrain, ytrain)

In [56]:
y_pred = final.predict_proba(Xtest)

In [59]:
y_final_prob = y_pred[:, 1] > BestThr

In [60]:
y_final_prob

array([ True,  True,  True, False,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True, False,  True, False,  True,  True,  True,
        True, False,  True, False,  True,  True,  True,  True, False,
        True, False,  True,  True,  True,  True,  True,  True,  True,
        True, False,  True,  True, False,  True,  True,  True,  True,
        True,  True,  True,  True,  True, False,  True,  True,  True,
       False,  True,  True,  True,  True,  True,  True,  True,  True,
       False,  True, False,  True,  True,  True, False,  True,  True,
       False, False,  True,  True,  True,  True,  True,  True,  True,
        True, False, False, False,  True,  True, False,  True,  True,
       False, False, False,  True,  True,  True,  True, False,  True,
       False,  True,  True, False,  True,  True, False,  True,  True,
        True,  True,  True,  True,  True, False, False,  True, False,
        True,  True,

**Вопрос:** какое значение recall получилось на тестовых данных после подбора порога?  
Ответ округлите до сотых.

При помощи подбора порога удалось сильно увеличить значение *recall*!  
Однако, как видно, на тесте не удалось сохранить условие $accuracy \geq 0.6$ (но в допустимые рамки уложились!) 

Это свидетельство небольшого переобучения модели. Однако в сухом остатке имеет значительное увеличение полноты, что является приоритетом для заказчика.

In [61]:
new_recall = recall_score(ytest, y_final_prob)

In [62]:
new_recall

0.8040540540540541

In [63]:
new_accuracy = accuracy_score(ytest, y_final_prob)

In [64]:
new_accuracy

0.5666666666666667